In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed


In [ ]:
import logging
logger = logging.getLogger('my_logger')

logging.basicConfig()

logger.debug('This is my 😂 debug message ')
logger.info('This is my 💜 info message ')
logger.warning('This is my 🤔 warning message ')
logger.error('This is my error 😱message ')
logger.critical('This is my 😭 critical message ')

ERROR:my_logger:This is my error 😱message 
CRITICAL:my_logger:This is my 😭 critical message 


In [ ]:
import os
import torch
import time


In [ ]:
torch.cuda.is_available()

True

In [ ]:
from multiprocessing import Pool, set_start_method

In [ ]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In case you only want to try with a small piece of text

In [ ]:
text = '''Wir haben durch de» Mund des Vorredners (Dumreicher) eben
gehört, was für Elemente sich in den Ministerien befinden. Ich weiß
nicht, wie Viele heute noch im Uuterrichtsiiünisterium sich vorsindc»,
die, wenn sie heute ihre Posten verlassen würden oder müßten, alle
wahrscheinlich ihren Platz dort (links) aufsuchen würden und die uns,
von Haß erfüllt und erregt, Beschwerden zuschicken als Beweis ihrer
gerechten Gesinnung für das slavisch« Element. (So ist es! reckts.)
Wenn wir eine solche Bureaukratie haben, dann können wir aller
dings nicht erwarten, daß unser« Anliegen in gerechter, unparteiischer
Weise erledigt werden (So ist es! rechts), und wenn dann gar so
ein Cherusker dazukommt und in dem böhmischen Schulwesen zu
hausen beginnt mit seinem Furor tentooicus — dann Gnade den
böhmischen Schulen! (Bravo! rechts.) Redner kommt nun auf die
Autonomie der Länder in Bezug auf das Volksschulwesen zu sprechen
und bemerkt: Herr v. Schmerling war in dieser Beziehung viel
autonomistischer und viel liberaler, als alle ktaatsinänner, die nach
ihm das Unterrichtsministerium leiteten, viel autonomistischer, ver
fassungsmäßiger und correcter, als die Reichsräthe, die hier Legis
lation wachten. Das Reichs-Vvlksschulgesetz ist in ganz incorrecter
Weise entstanden und stellt eine» Bruck der Verfassung dar (Bravo!
Bravo! rechts), und wir und das Ministerium befinden uns bei
Fefthaltung dieses Gesetzes in eine» unausgesetzten, in einem
perennirenden VerfaffungSbruche (Sehr gut! rechts), denn wenn die
Verfassung eine Wahrheit ist, so kann dieses Schulgesetz nicht aufrecht
bestehe«, es muß abgeändert werden, es muß de» Ländern und Völkern
die Autonomie in Bezug auf das Schulwesen wieder zurückgegeben
werden, denen man «s rnc«rerterwrise entzogen hat. (Beifall rechts.)
'''

- problems around the '<-pad->' - tends to remove some words


Frpm the original notebook - just to test out the model without padding

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/hmbyt_ONB_150_no_overlap_0124.model")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/hmbyt_ONB_150_no_overlap_0124.model")#.to("cuda")
if torch.cuda.is_available():
    model = model.to('cuda')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.model_max_length

150

In [ ]:
def chunk_text(text, max_tokens, tokenizer):
    logging.info("Starting text chunking")
    text = text.replace('\n', ' ')
    sentences = re.split(r'(?<=[.?!;])\s', text)
    chunks = []
    current_chunk = ""
    current_chunk_token_count = 0

    for sentence in sentences:
        sentence = sentence.strip()
        sentence_tokens = tokenizer.encode(sentence, add_special_tokens=False)
        logging.debug(f"Processing sentence: {sentence[:30]}...")

        if len(sentence_tokens) > max_tokens:
            words = sentence.split()
            for word in words:
                word_tokens = tokenizer.encode(word, add_special_tokens=False)
                if current_chunk_token_count + len(word_tokens) <= max_tokens:
                    current_chunk += (word + " ")
                    current_chunk_token_count += len(word_tokens)
                else:
                    chunks.append(current_chunk.strip())
                    current_chunk = word + " "
                    current_chunk_token_count = len(word_tokens)
        else:
            if current_chunk_token_count + len(sentence_tokens) <= max_tokens:
                current_chunk += (sentence + " ")
                current_chunk_token_count += len(sentence_tokens)
            else:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + " "
                current_chunk_token_count = len(sentence_tokens)

    if current_chunk.strip():
        chunks.append(current_chunk.strip())

    return chunks

In [ ]:
chunks = chunk_text(text, 150, tokenizer)
chunks

NameError: name 'text' is not defined

In [ ]:
def process_text_file(text, max_tokens=150, tokenizer=tokenizer,model=model):
    logging.info("Starting processing of text file")
    chunks = chunk_text(text, max_tokens, tokenizer)
    processed_text = ''

    for i, chunk in enumerate(chunks):
        logging.debug(f"Processing chunk {i+1}/{len(chunks)}")
        inputs = tokenizer(chunk, return_tensors="pt").input_ids

        if torch.cuda.is_available():
            inputs = inputs.to('cuda')

        outputs = model.generate(inputs, max_new_tokens=1000, do_sample=True, top_p=0.95)
        processed_chunk = tokenizer.decode(outputs[0], skip_special_tokens=True)

        processed_text += processed_chunk.strip() + " "

    processed_text = processed_text.rstrip()

    logging.info("Text file processing complete")
    return processed_text


In [ ]:
print(process_text_file(text,max_tokens=150, tokenizer=tokenizer,model=model))

Wir haben durch den Mund des Vorredners (Dumreicher) eben gehört, was für Elemente sich in den Ministerien befinden. Ich weiß nicht, wie Viele heute noch im Unterrichtsministerium sich vorsinden, die, wenn sie heute ihre Posten verlassen würden oder müßten, alle wahrscheinlich ihren Platz dort (links) aufsuchen würden und die uns, von Haß erfüllt und erregt, Beschwerden zuschicken als Beweis ihrer gerechten Gesinnung für das slavische Element. (So ist es! rechts.) Wenn wir eine solche Bureaukratie haben, dann können wir aller- dings nicht erwarten, daß unsere Anliegen in gerechter, unparteiischer Weise erledigt werden (So ist es! rechts), und wenn dann gar so ein Cherusker dazukommt und in dem böhmischen Schulwesen zu hausen beginnt mit seinem Furor tentonicus — dann Gnade den böhmischen Schulen! (Bravo! rechts.) Redner kommt nun auf die Autonomie der Länder in Bezug auf das Volksschulwesen zu sprechen und bemerkt: Herr v. Schmerling war in dieser Beziehung viel autonomistischer und vi

Now I split all relevant newspapers into smallers chunks, where each chunk is one row of the 'text' column in a dataframe


for the dataframe


In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('drive/MyDrive/nfp_vtl_analysis/csvs/nfp/czechs_topic3_nfp.csv')


In [ ]:
df

,Unnamed: 0,Document,Topic,Name,periodical,date,preceding_document,following_document
0,1553,"— Herrn V. E. S., Zglau und Herrn C. V., Prag:...",3,3_czechen_polen_deutschen_czechischen,nfp,1864-10-01,Bitten=Portoersatz von 12 kr.,Bitten um 12 kr. — Herrn Th. Boeldorf: Den Rom...
1,1789,"Wahrlich, wenn jene „Herren in Wien“ welche da...",3,3_czechen_polen_deutschen_czechischen,nfp,1864-11-01,"Wer sollten denn die „Herren in Wien“ sein, au...",Mit wahrer Pharisäer=Miene führen die hiesigen...
2,1790,Mit wahrer Pharisäer=Miene führen die hiesigen...,3,3_czechen_polen_deutschen_czechischen,nfp,1864-11-01,"Wahrlich, wenn jene „Herren in Wien“ welche da...","Dahinaber soll es, will's Gott, nicht kommen!"
3,1792,— Der vollzogene Minister=Wechsel gibt auch de...,3,3_czechen_polen_deutschen_czechischen,nfp,1864-11-01,"Dahinaber soll es, will's Gott, nicht kommen!",Graf Rechberg stand der czechischen Nation nic...
4,1793,Graf Rechberg stand der czechischen Nation nic...,3,3_czechen_polen_deutschen_czechischen,nfp,1864-11-01,— Der vollzogene Minister=Wechsel gibt auch de...,"Der Staatsminister, sagen die czechischen Orga..."
...,...,...,...,...,...,...,...,...
2738,310760,"August d. I. beschlossenen Programme, welches ...",3,3_czechen_polen_deutschen_czechischen,nfp,1900-12-01,Getreu demaus dem demokratischen Parteitage vo...,"DerAufruf erinnert auch daran, daß die, polnis..."
2739,310761,"DerAufruf erinnert auch daran, daß die, polnis...",3,3_czechen_polen_deutschen_czechischen,nfp,1900-12-01,"August d. I. beschlossenen Programme, welches ...",(Czechische Don Quixotia«den.)
2740,310762,(Czechische Don Quixotia«den.),3,3_czechen_polen_deutschen_czechischen,nfp,1900-12-01,"DerAufruf erinnert auch daran, daß die, polnis...",Wir erhalten folgende Zuschrift: „Sehr geehrte...
2741,310764,"v. Wessely ruft den Unmuth der Czechen hervor,...",3,3_czechen_polen_deutschen_czechischen,nfp,1900-12-01,Wir erhalten folgende Zuschrift: „Sehr geehrte...,"Desgleichen ist cs nichtwahr, daß die „Moravsk..."


In [ ]:
def process_dataframe(df,output_csv, max_tokens=150,tokenizer=None, model=None):
    print("Starting processing of DataFrame")

    for index, row in df.iterrows():
        text = row['Document']
        processed_text = process_text_file(text, max_tokens, tokenizer, model)


        df.at[index, 'processed_text'] = processed_text

    print("Completed processing DataFrame")

    df.to_csv(output_csv, encoding='utf-8')
    print("DataFrame saved to pickle file")





In [ ]:
output_csv = 'drive/MyDrive/nfp_vtl_analysis/csvs/nfp/czechs_topic3_nfp_pp.csv'

In [ ]:
process_dataframe(df, output_csv, max_tokens=150, tokenizer=tokenizer, model=model)



Starting processing of DataFrame
Completed processing DataFrame
DataFrame saved to pickle file


for the whole file

In [ ]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content



In [ ]:
def write_file(content, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)


In [ ]:
def process_file(input_file_path, output_file_path, max_tokens=150, tokenizer=tokenizer, model=model):
    text = read_file(input_file_path)
    processed_text = process_text_file(text, max_tokens, tokenizer)
    write_file(processed_text, output_file_path)


In [ ]:
'''
input_file_path = '/content/vtl18600915_merged.txt'
output_file_path = '/content/vtl18600915_postprocessed.txt'

process_file(input_file_path, output_file_path, max_tokens=150, tokenizer=tokenizer, model=model)
'''

"\ninput_file_path = '/content/vtl18600915_merged.txt'  # Replace with your input file path\noutput_file_path = '/content/vtl18600915_postprocessed.txt'  # Replace with your desired output file path\n\n# Process the input file and save the result to the output file\nprocess_file(input_file_path, output_file_path, max_tokens=150, tokenizer=tokenizer, model=model)\n"

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

all files in a folder


In [ ]:
def list_txt_files(folder_path):
    return [f for f in os.listdir(folder_path) if f.endswith('.txt')]


In [ ]:
source_folder = '/content/drive/MyDrive/test'  # Replace with your source folder path
target_folder = '/content/drive/MyDrive/test_nfp_pp'  # Replace with your target folder path


In [ ]:
def setup_logging():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')


In [ ]:
def process_all_files(source_folder, target_folder, max_tokens=150, tokenizer=tokenizer, model=model):
    setup_logging()
    print("Starting processing of all files")


    os.makedirs(target_folder, exist_ok=True)


    files = list_txt_files(source_folder)
    print(f"Found {len(files)} files to process")

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_file = {executor.submit(process_file, os.path.join(source_folder, file), os.path.join(target_folder, file.replace('_merged.txt', '_postprocessed.txt')), max_tokens, tokenizer, model): file for file in files}
        print('file submitted')

        for future in as_completed(future_to_file):
            file = future_to_file[future]
            try:
                future.result()
                print(f"Successfully processed '{file}' and saved to '{target_folder}'")
            except Exception as exc:
                print(f"File {file} generated an exception: {exc}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"Completed processing all files. Total time taken: {total_time:.2f} seconds")



In [ ]:
process_all_files(source_folder, target_folder, max_tokens=150, tokenizer=tokenizer, model=model)

Starting processing of all files
Found 2 files to process
file submitted


KeyboardInterrupt: 

In [ ]:

# Process all .txt files in the source folder
process_all_files(source_folder, target_folder, max_tokens=150, tokenizer=tokenizer, model=model)
